In [149]:
# -*- coding: utf-8 -*-

import numpy as np
from scipy import special
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.datasets import make_classification
from scipy.special import expit

# Используйте scipy.special для вычисления численно неустойчивых функций
# https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special

In [150]:
def lossf(w, X, y, l1, l2):
    """
    Вычисление функции потерь.

    :param w: numpy.array размера  (M,) dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора 
    :param l2: float, l2 коэффициент регуляризатора 
    :return: float, значение функции потерь
    """
    lossf = np.sum(np.logaddexp(0, -y * X.dot(w))) + l1 * np.sum(np.abs(w)) + l2 * np.sum(w ** 2)
    return lossf

In [151]:
def gradf(w, X, y, l1, l2):
    """
    Вычисление градиента функции потерь.

    :param w: numpy.array размера  (M,), dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора 
    :param l2: float, l2 коэффициент регуляризатора 
    :return: numpy.array размера  (M,), dtype = np.float, градиент функции потерь d lossf / dw
    """
    gradw = (-y * expit(-y * X.dot(w))).dot(X) + l1 * np.sign(w) + 2 * l2*w
    return gradw

In [152]:
class LR(ClassifierMixin, BaseEstimator):
    def __init__(self, lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0):
        """
        Создание класса для лог регрессии
        
        :param lr: float, длина шага для оптимизатора
        :param l1: float, l1 коэффициент регуляризатора 
        :param l2: float, l2 коэффициент регуляризатора
        :param num_iter: int, число итераций оптимизатора
        :param verbose: bool, ключик для вывода
        """
        self.l1 = l1
        self.l2 = l2
        self.w = None
        self.lr = lr
        self.verbose = verbose
        self.num_iter = num_iter
        
    def fit(self, X, y):
        """
        Обучение логистической регрессии.
        Настраивает self.w коэффициенты модели.

        Если self.verbose == True, то выводите значение 
        функции потерь на итерациях метода оптимизации. 

        :param X: numpy.array размера  (N, M), dtype = np.float
        :param y: numpy.array размера  (N,), dtype = np.int
        :return: self
        """
        n, d = X.shape
        step_size = self.lr
        self.w = np.ones(d)
        for step in np.arange(self.num_iter):
            if self.verbose:
                print lossf(self.w, X, y, self.l1, self.l2)
            self.w = self.w - gradf(self.w, X, y, self.l1, self.l2) * step_size
        return self
        
    def predict_proba(self, X):
        """
        Предсказание вероятности принадлежности объекта к классу 1.
        Возвращает np.array размера (N,) чисел в отрезке от 0 до 1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return: numpy.array размера  (N,), dtype = np.float
        """
        # Вычислите вероятности принадлежности каждого 
        # объекта из X к положительному классу, используйте
        # эту функцию для реализации LR.predict
        probs = expit(X.dot(self.w))
        return probs
    
    def predict(self, X):
        """
        Предсказание класса для объекта.
        Возвращает np.array размера (N,) элементов 1 или -1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return:  numpy.array размера  (N,), dtype = np.int
        """
        # Вычислите предсказания для каждого объекта из X
        predicts = np.sign(self.predict_proba(X) - 0.5)
        return predicts 

In [153]:
def test_work():
    print "Start test"
    X, y = make_classification(n_features=100, n_samples=1000)
    y = 2 * (y - 0.5)

    try:
        clf = LR(lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0)
    except Exception:
        assert False, "Создание модели завершается с ошибкой"
        return

    try:
        clf = clf.fit(X, y)
    except Exception:
        assert False, "Обучение модели завершается с ошибкой"
        return

    assert isinstance(lossf(clf.w, X, y, 1e-3, 1e-3), float), "Функция потерь должна быть скалярной и иметь тип np.float"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).shape == (100,), "Размерность градиента должна совпадать с числом параметров"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).dtype == np.float, "Вектор градиента, должен состоять из элементов типа np.float"
    assert clf.predict(X).shape == (1000,), "Размер вектора предсказаний, должен совпадать с количеством объектов"
    assert np.min(clf.predict_proba(X)) >= 0, "Вероятности должны быть не меньше, чем 0"
    assert np.max(clf.predict_proba(X)) <= 1, "Вероятности должны быть не больше, чем 1"
    assert len(set(clf.predict(X))) == 2, "Метод предсказывает больше чем 2 класса на двух классовой задаче"
    print "End tests"

In [154]:
test_work()

Start test
End tests


In [155]:
from sklearn.linear_model import LogisticRegression

In [156]:
lr_sklearn = LogisticRegression(penalty='l2', random_state=42, max_iter=1000)

In [157]:
my_lr = LR(lr=0.1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0)

In [158]:
X, Y = make_classification(n_features=100, n_samples=2000)
Y = 2 * (Y - 0.5)

In [159]:
lr_sklearn.fit(X_train, Y_train)
my_lr.fit(X_train, Y_train)

LR(l1=0.0001, l2=0.0001, lr=0.1, num_iter=1000, verbose=0)

In [160]:
from sklearn.model_selection import cross_val_score

In [163]:
%%time
cross_val_score(lr_sklearn, X, Y).mean()

CPU times: user 64.4 ms, sys: 2.46 ms, total: 66.9 ms
Wall time: 68.1 ms


0.90150345247796526

In [164]:
%%time
cross_val_score(my_lr, X, Y).mean()

CPU times: user 485 ms, sys: 85.8 ms, total: 571 ms
Wall time: 509 ms


0.84151918034976514